In [1]:
# import libraries
from selenium import webdriver
from scrapy import Selector
import requests
import pandas as pd



In [2]:
# define parameters for url, get requests, and selector

# to remove column limit; otherwise some data could be lost
pd.set_option('max_colwidth',500)   

# indeed.com url for data scientist in scottsdale arizona, sorted by date
base_url = 'https://www.indeed.com/jobs?q=data+scientist&l=Scottsdale%2C+AZ&sort=date'

# start page #
start_from = '&start='

# set initial page to 1
page = 1

# will need to delete beleo if already used in main loop
url = base_url + start_from + str(page)

# Create the string html containing the HTML source
html = requests.get( url = url).content
  
# create selector object
sel = Selector( text=html ) 




In [3]:
# MAIN LOOP

# initialize lists for each field 
job_titles = []
job_links = []
job_postings = []
company_names = []
company_links = []
locations = []
summary = []
ratings = []

# initialize pandas dataframe
df = pd.DataFrame()

# loop through pages 1 - 20 (indeed refers to them in multiples of 10 )
for page in range(1, 5):
    # indeed refers to pages in multiples of 10
    page = (page - 1) * 10
    # combined url with parameters
    url = base_url + start_from + str(page)
    # printed the urls to confirm they were valid web address
    print(url)

    # created the html containing the HTML source
    html = requests.get( url = url).content
  
    # created selector object
    sel = Selector( text=html ) 
        
    # pull job titles and append to list   
    job_titles.append(sel.xpath('//div[@class="title"]/a/@title').extract())
  
    # pull job links and append to list
    job_links.append(sel.xpath('//div[@class="title"]/a/@href').extract())

    # pull job posting(date) and append to list
    job_postings.append(sel.xpath('//span[@class="date "]/text()').extract())

    # pull company_names and append to list   
    company_names.append(sel.xpath('//span[@class="company"]/a/text()').extract())

    # pull company_links and append to list  
    company_links.append(sel.xpath('//span[@class="company"]/a/@href').extract())        # print(company_links)
   
    # pull locations and append to list
    locations.append(sel.css('div.sjcl > span::text').extract())
    
    # pull sumamry and append
    summary.append(sel.xpath('//div[@class="summary"]//text()').extract())

    # pull ratings and append to list
    ratings.append(sel.xpath('//span[@class="ratingsContent"]/text()').extract())
  
    

https://www.indeed.com/jobs?q=data+scientist&l=Scottsdale%2C+AZ&sort=date&start=0
https://www.indeed.com/jobs?q=data+scientist&l=Scottsdale%2C+AZ&sort=date&start=10
https://www.indeed.com/jobs?q=data+scientist&l=Scottsdale%2C+AZ&sort=date&start=20
https://www.indeed.com/jobs?q=data+scientist&l=Scottsdale%2C+AZ&sort=date&start=30


In [4]:
# initialize lists to hold header information and combined list data
data = []
header_list = []

# zip together separate lists for each field
data = zip(job_titles, job_links, job_postings, company_names, company_links, locations, summary, ratings)

# add header list to idenftiy fields
header_list = ['job_titles', 'job_links', 'job_postings', 'company_names', 'company_links', 'locations', 'summary', 'ratings']
         
# zip together header and data lists    
zip_list = zip(header_list, data)

# unpack headers and data from zipped list
headers, data = zip(*zip_list)

# unpack data fields from zipped list
job_titles, job_links, job_postings, company_names, company_links, locations, summary, ratings = zip(*data)


In [5]:
# create pandas dataframe structured as a dictionary 
df = pd.DataFrame({'company_name': company_names, 'job_title': job_titles, 
                'job_link': job_links, 'job_posted': job_postings,
                'company_link': company_links, 'job_location': locations,
                'job_summary' : summary, "ratings" : ratings,
                'overall_rating': None, 'wl_bal_rating': None, 
                'benefit_rating': None, 'jsecurity_rating': None, 
                'mgmt_rating': None, 'culture_rating': None,
                'reviews_count' : None
                })

In [6]:
print(df)

                                                                                                                                                                                                                              company_name  \
0               [\nUSAA, \nUSAA, \nWells Fargo Bank, \nUSAA, \nCircle K, \nCircle K, \nTransPerfect, \nDiscover Financial Services, \nNationwide Mutual Insurance Company, \nAmerican Express, \nLighthouse, \nShamrock Foods, \nDeloitte]   
1  [\nUSAA, \nDeloitte, \nUSAA, \nLifesprk, \nHealth Services Advisory Group, Inc., \nWipro Limited, \nAmerican Express, \nFreeport McMoRan, \nUSAA, \nDiscount Tire Corporate Careers, \nNationwide Mutual Insurance Company, \nFreestar]   
2                                                                                                          [\nUSAA, \nUSAA, \nLifesprk, \nAlbertsons Companies, \nUSAA, \nMcKesson, \nUSAA, \nBNP Paribas, \nAmerican Express, \nDeloitte]   
3               [\nUSAA, \nUSAA, \nLifesprk, \nT

In [7]:
# initialize list to hold links to company websites 
url_list = []

# add url prefix to link
pre_url = 'http://www.indeed.com/cmp/'
# add reviews to end of link

# iterate through all company review links for all companies in first list
for list in df['company_name']:
    # iterate through each line
    for line in list:
        # iterate through each line
        # confirm there is data in field
        if len(line) > 0: 
            
            # create full url by concatentaing strings (stripping whitespace from company name)
            full_url = pre_url + line.replace(' ', '-').strip()
                
            # strip whitespace from link
            full_url.strip()
    
            # print full_url to confirm valid website
            # print(full_url)
                
            # Create a second html for each company reveiw website
            html2 = requests.get( full_url ).content

            # Create the Selector object set from html
            sel2 = Selector( text=html2 )
                       
            # use selector list to pull all company ratings and review count              
            comp_rating_overall = sel2.css('.cmp-CompactHeaderCompanyRatings-value::text').extract()
            wl_rating = sel2.xpath('//*[@data-tn-element="review-filter-wlbalance"]/*[@class="cmp-TopicFilter-rating"]/text()').extract()
            benefits_rating = sel2.xpath('//*[@data-tn-element="review-filter-paybenefits"]/*[@class="cmp-TopicFilter-rating"]/text()').extract()
            sec_rating = sel2.xpath('//*[@data-tn-element="review-filter-jobsecadv"]/*[@class="cmp-TopicFilter-rating"]/text()').extract()
            mgmt_rating = sel2.xpath('//*[@data-tn-element="review-filter-mgmt"]/*[@class="cmp-TopicFilter-rating"]/text()').extract()
            culture_rating = sel2.xpath('//*[@data-tn-element="review-filter-culture"]/*[@class="cmp-TopicFilter-rating"]/text()').extract()
            reviews_count = sel2.css('.cmp-RatingsCountLink::text').extract()
            
        else:
            print('None')
    


In [8]:
# print head of dataframe
print(df.head)

<bound method NDFrame.head of                                                                                                                                                                                                                               company_name  \
0               [\nUSAA, \nUSAA, \nWells Fargo Bank, \nUSAA, \nCircle K, \nCircle K, \nTransPerfect, \nDiscover Financial Services, \nNationwide Mutual Insurance Company, \nAmerican Express, \nLighthouse, \nShamrock Foods, \nDeloitte]   
1  [\nUSAA, \nDeloitte, \nUSAA, \nLifesprk, \nHealth Services Advisory Group, Inc., \nWipro Limited, \nAmerican Express, \nFreeport McMoRan, \nUSAA, \nDiscount Tire Corporate Careers, \nNationwide Mutual Insurance Company, \nFreestar]   
2                                                                                                          [\nUSAA, \nUSAA, \nLifesprk, \nAlbertsons Companies, \nUSAA, \nMcKesson, \nUSAA, \nBNP Paribas, \nAmerican Express, \nDeloitte]   
3               [\

In [9]:
# data cleaning
# add indeed prefix to job link and company link
prefix = 'http://www.indeed.com'


# iterate tthrough list in company_link
for list in df['company_link']:
    for line in list:
        # strip whitespace and page breaks
        company_link_clean = prefix + line 
        # add new column to data frame for cleaned name
        df['company_link_clean'] = company_link_clean
        print(company_link_clean)


for job in df['job_link']:
    for detail in job:
        # strip whitespace and page breaks
        job_link_clean = prefix + detail 
        # add new column to data frame for cleaned name
        df['job_link_clean'] = job_link_clean
        print(job_link_clean)


http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Wells-Fargo
http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Circle-K
http://www.indeed.com/cmp/Circle-K
http://www.indeed.com/cmp/Transperfect-983d8371
http://www.indeed.com/cmp/Discover-Financial-Services
http://www.indeed.com/cmp/Nationwide-Mutual-Insurance-Company
http://www.indeed.com/cmp/American-Express
http://www.indeed.com/cmp/Lighthouse
http://www.indeed.com/cmp/Shamrock-Foods-Company
http://www.indeed.com/cmp/Deloitte
http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Deloitte
http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Lifesprk
http://www.indeed.com/cmp/Health-Services-Advisory-Group,-Inc.
http://www.indeed.com/cmp/Wipro
http://www.indeed.com/cmp/American-Express
http://www.indeed.com/cmp/Freeport--mcmoran
http://www.indeed.com/cmp/Usaa
http://www.indeed.com/cmp/Discount-Tire
http://www.indeed.com/cmp/Nationwide-Mutual-Insurance-Company
http://www.indeed.com/cm

In [10]:
# DATA CLEANING
# remove whitspace/page breaks from company name and ratings
    
 
# iterate through company names
for list in df['company_name']:
    for line in list:
        # strip whitespace and page breaks from
        company_name_clean = line.lstrip('\n')
        # add new column to data frame for cleaned name
        df['company_name_clean'] = company_name_clean
        print(company_name_clean)
        

for rating in df['ratings']:
    for detail in rating:
        ratings_clean = detail.replace('\n', '')
        df['ratings_clean'] = ratings_clean
        print(ratings_clean)



USAA
USAA
Wells Fargo Bank
USAA
Circle K
Circle K
TransPerfect
Discover Financial Services
Nationwide Mutual Insurance Company
American Express
Lighthouse
Shamrock Foods
Deloitte
USAA
Deloitte
USAA
Lifesprk
Health Services Advisory Group, Inc.
Wipro Limited
American Express
Freeport McMoRan
USAA
Discount Tire Corporate Careers
Nationwide Mutual Insurance Company
Freestar
USAA
USAA
Lifesprk
Albertsons Companies
USAA
McKesson
USAA
BNP Paribas
American Express
Deloitte
USAA
USAA
Lifesprk
Tivity Health, Inc.
Wells Fargo
Insight Enterprises, Inc.
Deloitte
Deloitte
Brightside
ViaSat
Statistics & 
 Corporation (SDC)
MUFG
Cable One Inc.
Deloitte
3.9

3.9

3.7

3.9

3.3

3.3

3.0

3.9

3.9

4.1

3.7

3.2

4.0

3.9

4.0

3.9

3.5

3.0

3.8

4.1

4.1

3.9

4.0

3.9

4.3

3.9

3.9

3.5

3.7

3.9

3.6

3.9

3.9

4.1

4.0

3.9

3.9

3.5

3.0

3.7

3.6

4.0

4.0

3.4

3.8

3.6

3.5

3.8

4.0

